In [ ]:
from typing import List
import xmlrpc.client
import socket
from bs4 import BeautifulSoup
import requests
from pathlib import Path


class RPCSender(object):
    def __init__(self, server_url: str, token: str, download_root_path: str):
        self.server_proxy = xmlrpc.client.ServerProxy(server_url)
        self.token = token
        self.download_root_path = download_root_path
        self.test_connection()

    def test_connection(self):
        try:
            version = self.server_proxy.aria2.getVersion("token:" + self.token)
            print(f"Connected to server. Version: {version['version']}")
        except socket.timeout:
            print("Connection to server timed out.")
        except Exception as e:
            print(f"Failed to connect to server: {e}")

    def download_url(self, url: str):
        options = {"dir": f"{self.download_root_path}/{Path(url).parent.parent.parent.name}",
                   "out": Path(url).name.split("?download=true")[0]}
        gid = self.server_proxy.aria2.addUri("token:" + self.token, [url], options)
        print(f"Download started with GID: {gid}")

    def download_urls(self, urls: List[str]):
        for url in urls:
            self.download_url(url)


def get_hub(url):
    url_cut = url.split('https://huggingface.co')[-1].replace('tree', 'resolve')
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    file_links = []

    for link in soup.find_all("a"):
        href = link.get("href")
        if href.startswith(url_cut):
            file_links.append(f"https://huggingface.co{href}")
    for fl in file_links:
        print(fl)
    return file_links

In [ ]:
server_url = "http://localhost:19628/rpc"
token = '113113'
root_path = "f:/downloads"
model_id = "THUDM/chatglm2-6b"
rpc = RPCSender(server_url, token, download_root_path=root_path)

In [ ]:
download_urls = get_hub(f"https://huggingface.co/{model_id}/tree/main")

In [ ]:
rpc.download_urls(urls=download_urls)

In [ ]:
# 选择下载
rpc.download_urls(urls=[
    # "https://huggingface.co/MaziyarPanahi/gemma-2b-it-GGUF/resolve/main/gemma-2b-it.Q2_K.gguf?download=true",
    "https://huggingface.co/MaziyarPanahi/gemma-2b-it-GGUF/resolve/main/gemma-2b-it.Q3_K_M.gguf?download=true",
    "https://huggingface.co/MaziyarPanahi/gemma-2b-it-GGUF/resolve/main/gemma-2b-it.Q4_K_M.gguf?download=true"
])